In [10]:
# Expecting at this point we already have 
# 1. Feat.npy generated, 2. corresponding csv files 3. train/test races .txt file
import os
import glob
import cv2
import copy, time

import torch
import numpy as np

from util.train_helper import *
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Set up Parameters
# HVT small batch for testing
# train_races_txt = "G:/tim/Project/Connectivity_train_test/HVT_small_train_races.txt"
# test_races_txt = "G:/tim/Project/Connectivity_train_test/HVT_small_test_races.txt"
# trip_feat_train_dir = "D:/Bitbucket/triplettraining/feat/HVT_train"
# trip_feat_test_dir = "D:/Bitbucket/triplettraining/feat/HVT_test"
# cap_gt_dir = "G:/tim/Project/Connectivity_train_test/real_data/HVT-2018-2019_full_recall"

# HVT Full
# train_races_txt = "G:/tim/Project/Connectivity_train_test/HVT_train_races.txt"
# test_races_txt = "G:/tim/Project/Connectivity_train_test/HVT_test_races.txt"
# trip_feat_train_dir = "D:/Bitbucket/triplettraining/feat/HVT_train"
# trip_feat_test_dir = "D:/Bitbucket/triplettraining/feat/HVT_test"
# cap_gt_dir = "G:/tim/Project/Connectivity_train_test/real_data/HVT-2018-2019_full_recall"

# Kranji Full
# train_races_txt = "G:/tim/Project/Connectivity_train_test/Kranji_train_races.txt"
# test_races_txt = "G:/tim/Project/Connectivity_train_test/Kranji_test_races.txt"
# trip_feat_train_dir = "D:/Bitbucket/triplettraining/feat/Kranji_train"
# trip_feat_test_dir = "D:/Bitbucket/triplettraining/feat/Kranji_test"
# cap_gt_dir = "G:/tim/Project/Connectivity_train_test/real_data/Kranji-2018_verified_26races"

# STT Full
# train_races_txt = "D:/Bitbucket/HorseRacing/HorseRacingTrainingAnalysis/connectivitytraining/data/STT_train_races.txt"
# test_races_txt = "D:/Bitbucket/HorseRacing/HorseRacingTrainingAnalysis/connectivitytraining/data/STT_test_races.txt"
# trip_feat_train_dir = "D:/Bitbucket/HorseRacing/HorseRacingTrainingAnalysis/triplettraining/data/feat/STT_train"
# trip_feat_test_dir = "D:/Bitbucket/HorseRacing/HorseRacingTrainingAnalysis/triplettraining/data/feat/STT_test"
# cap_gt_dir = "G:/tim/Project/Connectivity_train_test/real_data/STT_2017_2018-NotVerified-revised"

# STAWT Full
train_races_txt = "D:/Bitbucket/HorseRacing/HorseRacingTrainingAnalysis/connectivitytraining/data/STAWT_train_races.txt"
test_races_txt = "D:/Bitbucket/HorseRacing/HorseRacingTrainingAnalysis/connectivitytraining/data/STAWT_test_races.txt"
trip_feat_train_dir = "D:/Bitbucket/HorseRacing/HorseRacingTrainingAnalysis/triplettraining/data/feat/STAWT_train"
trip_feat_test_dir = "D:/Bitbucket/HorseRacing/HorseRacingTrainingAnalysis/triplettraining/data/feat/STAWT_test"
cap_gt_dir = "G:/tim/Project/Connectivity_train_test/real_data/STAWT_2017_2018-NotVerified-revised"

param_dict = get_default_param()

batch_size = 128
track = "STAWT"
time_str = param_dict["time_str"]
proj_name = f"Connectivity_batch_{batch_size}_{track}_{time_str}"
param_dict["project_name"] = proj_name
param_dict["train_batch_size"] = batch_size
param_dict["test_batch_size"] = batch_size

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Construct the dataset from the triplets files
from datasets.dataset import ConnDataset
from torch.utils.data import Dataset, TensorDataset, DataLoader, WeightedRandomSampler
conn_train_dataset = ConnDataset(train_races_txt, trip_feat_train_dir, cap_gt_dir)
conn_test_dataset = ConnDataset(test_races_txt, trip_feat_test_dir, cap_gt_dir)

# This part is to balance the data, 
# to ensure even distribution of the data within batch
# If WeightedRandomSampler is being used, maximum 2**24 data is limited,
# Breaking down the dataset into smaller part?
labels = conn_train_dataset.label_classes
label_unique, counts = np.unique(labels, return_counts = True)
class_weights = [sum(counts) / c for c in counts]
example_weights = [class_weights[e] for e in labels]
train_sampler = WeightedRandomSampler(example_weights, len(labels))

labels = conn_test_dataset.label_classes
label_unique, counts = np.unique(labels, return_counts = True)
class_weights = [sum(counts) / c for c in counts]
example_weights = [class_weights[e] for e in labels]
test_sampler = WeightedRandomSampler(example_weights, len(labels))

train_dataloader = DataLoader(conn_train_dataset, sampler = train_sampler, batch_size = param_dict['train_batch_size'])
test_dataloader = DataLoader(conn_test_dataset, sampler = test_sampler, batch_size = param_dict['val_batch_size'])

Total 84 races to be processed.
Processing STAWT-20170910-06 (1 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20170910-10 (2 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20170916-03 (3 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20170916-05 (4 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20170924-02 (5 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20170924-06 (6 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20170924-08 (7 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20171008-07 (8 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20171008-09 (9 / 84)...
This Race exceeds max sample pe

This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180530-08 (74 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180610-02 (75 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180616-02 (76 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180624-04 (77 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180624-09 (78 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180701-01 (79 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180701-05 (80 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180701-09 (81 / 84)...
This Race exceeds max sample per race: 199728 will be randomly sampled
Processing STAWT-20180708

In [12]:
from model.conn import ConnNet

conn_net = ConnNet().to(param_dict['device'])

dataloaders = {"train":train_dataloader, "val":test_dataloader}

loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(conn_net.parameters(), lr = 0.001, momentum = 0.9)

# Train Model
conn_net = train_model(conn_net, dataloaders, loss, optimizer, param_dict = param_dict)

Start training ...


Epoch 0/99
----------


val Loss: 0.6919 Acc: 0.6271
saving new best model, val_acc: 0.6271009606694291
train Loss: 0.0170 Acc: 0.9950
saving trained model


Epoch 1/99
----------


train Loss: 0.0087 Acc: 0.9976


...

train Loss: 0.0066 Acc: 0.9981


Epoch 98/99
----------


train Loss: 0.0066 Acc: 0.9981


Epoch 99/99
----------


val Loss: 0.3155 Acc: 0.9982
train Loss: 0.0066 Acc: 0.9981


In [10]:
len(conn_test_dataset.inputs)

2560399